<a href="https://colab.research.google.com/github/aTunass/Cosine_similarity/blob/main/Text_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Dataset

In [4]:
!pip install datasets==2.13.1
#Download MS MACRO Dataset

In [5]:
from datasets import load_dataset

dataset = load_dataset('ms_marco', 'v1.1')
subset = dataset['test']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Dataset ms_marco downloaded and prepared to /root/.cache/huggingface/datasets/ms_marco/v1.1/1.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# Lấy tên các cột trong dataset
column_names = dataset['test'].column_names
print(column_names)
print(dataset['test'].info.description)
print(len(dataset['test']))
print(subset[:2])

['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers']

Starting with a paper released at NIPS 2016, MS MARCO is a collection of datasets focused on deep learning in search.

The first dataset was a question answering dataset featuring 100,000 real Bing questions and a human generated answer.
Since then we released a 1,000,000 question dataset, a natural langauge generation dataset, a passage ranking dataset,
keyphrase extraction dataset, crawling dataset, and a conversational search.

There have been 277 submissions. 20 KeyPhrase Extraction submissions, 87 passage ranking submissions, 0 document ranking
submissions, 73 QnA V2 submissions, 82 NLGEN submisions, and 15 QnA V1 submissions

This data comes in three tasks/forms: Original QnA dataset(v1.1), Question Answering(v2.1), Natural Language Generation(v2.1).

The original question answering datset featured 100,000 examples and was released in 2016. Leaderboard is now closed but data is availible below.

The 

In [11]:
queries_infos = []
queries = []
corpus = []

for sample in subset:
    query_type = sample['query_type']
    if query_type != 'entity':
        continue
    query_id = sample['query_id']
    query_str = sample['query']
    passages_dict = sample['passages']
    is_selected_lst = passages_dict['is_selected']
    passage_text_lst = passages_dict['passage_text']
    query_info = {
        'query_id': query_id,
        'query': query_str,
        'relevant_docs': []
    }
    current_len_corpus = len(corpus)
    for idx in range(len(is_selected_lst)):
        if is_selected_lst[idx] == 1:
            doc_idx = current_len_corpus + idx
            query_info['relevant_docs'].append(doc_idx)

    if query_info['relevant_docs'] == []:
        continue

    queries.append(query_str)
    queries_infos.append(query_info)
    corpus += passage_text_lst

In [14]:
print(queries[0])
print(queries_infos[0])
print(corpus[0])

what class are spiders in
{'query_id': 7, 'query': 'what class are spiders in', 'relevant_docs': [4]}
SUBPHYLUM CHELICERATA, CLASS ARACHNIDA. Spiders. This group contains many familiar organisms, including the spiders, mites, scorpions and ticks. Examine the large spider on the right. Again, notice that there are two body regions, a cephalothorax and an abdomen. On the cephalothorax are two to four pairs of simple eyes.


![](https://i.imgur.com/kNrf1DD.png![test.png])

# Token, Normalize, Vectorize function

In [27]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
def tokenize(text):
    return text.split()
nltk.download('stopwords')
english_stopwords = stopwords.words('english')
remove_chars = string.punctuation
stemmer = PorterStemmer()

def text_normalize(text):
    text = text.lower()
    for char in remove_chars:
        text = text.replace(char, '')
    text = ' '.join([word for word in tokenize(text) if word not in english_stopwords])
    text = ' '.join([stemmer.stem(word) for word in tokenize(text)])

    return text
def vectorize(text, dictionary):
    #khởi tạo dict
    word_count_dict = {word: 0 for word in dictionary}
    normalized_text = text_normalize(text)
    tokens = tokenize(normalized_text)
    for token in tokens:
        try:
            word_count_dict[token] += 1
        except:
            pass
    vector = list(word_count_dict.values())
    return vector
test = "Nguyenx Hoang Anh Tuan +-, * ///"
print(text_normalize(test))

nguyenx hoang anh tuan


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Create Dictionary

In [28]:
def create_dictionary(corpus):
    dictionary = []
    for doc in corpus:
        normalized_doc = text_normalize(doc)
        tokens = tokenize(normalized_doc)
        for token in tokens:
            if token not in dictionary:
                dictionary.append(token)

    return dictionary
%time
dictionary = create_dictionary(corpus)
print(dictionary[0])
print(len(dictionary))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 11 µs
subphylum
22111


# Create_doc_term_matrix

In [44]:
def create_doc_term_matrix(corpus, dictionary):
    doc_term_matrix = {}
    for idx, doc in enumerate(corpus):
        vector = vectorize(doc, dictionary)
        doc_term_matrix[(doc, idx)] = vector
    return doc_term_matrix
%time
doc_term_matrix = create_doc_term_matrix(corpus, dictionary)
print(len(doc_term_matrix))
print(doc_term_matrix[(corpus[0],0)])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.82 µs
7303
[1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [49]:
print(list(doc_term_matrix.items())[0]) #trả về list key_value, key là doc_infor, value là doc_vector

(('SUBPHYLUM CHELICERATA, CLASS ARACHNIDA. Spiders. This group contains many familiar organisms, including the spiders, mites, scorpions and ticks. Examine the large spider on the right. Again, notice that there are two body regions, a cephalothorax and an abdomen. On the cephalothorax are two to four pairs of simple eyes.', 0), [1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Ranking

In [42]:
my_dict = {
    "apple": 1,
    "banana": 2,
    "orange": 3,
}

items_list = my_dict.items()

# Truy cập cặp key-value trong dict_items mà không cần chuyển sang list
for key, value in items_list:
    print(f"Key: {key}, Value: {value}")

Key: apple, Value: 1
Key: banana, Value: 2
Key: orange, Value: 3


In [46]:
from scipy import spatial

def similarity(a, b):
    return 1 - spatial.distance.cosine(a, b)
def ranking(query, dictionary, doc_term_matrix):
    query_vec = vectorize(query, dictionary)
    scores = []
    for doc_info, doc_vec in doc_term_matrix.items():
        sim = similarity(query_vec, doc_vec)
        scores.append((sim, doc_info))
    scores.sort(reverse=True)

    return scores

# Run

In [48]:
query_lst = ['what is the official language in Fiji']
top_k = 10
for query in query_lst:
    scores = ranking(query, dictionary, doc_term_matrix)
    print(f'Query: {query}')
    print('=== Relevant docs ===')
    for idx in range(top_k):
        doc_score = scores[idx][0]
        doc_content = scores[idx][1][0]
        doc_id = scores[idx][1][1]

        print(f'Top {idx + 1}; Score: {doc_score:.4f}')
        print(doc_id)
        print('\n')
        print(doc_content)
        print('\n')

Query: what is the official language in Fiji
=== Relevant docs ===
Top 1; Score: 0.6556
6388


The official languages in Fiji are Fijian and English. A dialect of Hindustani is also widely spoken among Indo-Fijians.  _________________________________________   T … he official and everyday language of Fiji is English. Fijian and Fiji-Hindi are second languages in the island nation.


Top 2; Score: 0.6556
6386


The official languages in Fiji are Fijian and English. A dialect of Hindustani is also widely spoken among Indo-Fijians.  _________________________________________   T … he official and everyday language of Fiji is English. Fijian and Fiji-Hindi are second languages in the island nation.


Top 3; Score: 0.5715
6382


The official languages. Fiji’s 1997 Constitution established Fijian as one of the official languages of the country. Fijian is an Austronesian language, a grouping that includes thousands of other languages spanning the globe. The language is of the Malayo-Polynesian

In [51]:
print(list(doc_term_matrix.items())[6388][0])

('The official languages in Fiji are Fijian and English. A dialect of Hindustani is also widely spoken among Indo-Fijians.  _________________________________________   T … he official and everyday language of Fiji is English. Fijian and Fiji-Hindi are second languages in the island nation.', 6388)
